In [1]:
!pip install openai

In [2]:
!pip install google-api-python-client

In [17]:
import openai
import json
from google.oauth2 import service_account
import googleapiclient.discovery

In [18]:
# Load and set our OpenAI API key
openai.api_key = open("key.txt", "r").read().strip("\n")

In [19]:
# Load and set Google Calendar API credentials
credentials = service_account.Credentials.from_service_account_file('YOUR CREDENTIALS FILE NAME.json')
calendar_service = googleapiclient.discovery.build('calendar', 'v3', credentials=credentials)

In [20]:
def get_user_message():
    # Function to get user input
    return input("You: ")

In [21]:
def get_calendar_events(date):
    """Get the events on a given date from Google Calendar"""
    events_result = calendar_service.events().list(
        calendarId='primary',
        timeMin=f'{date}T00:00:00Z',
        timeMax=f'{date}T23:59:59Z',
        singleEvents=True,
        orderBy='startTime'
    ).execute()

    events = events_result.get('items', [])
    formatted_events = []
    for event in events:
        formatted_event = {
            "summary": event['summary'],
            "start_time": event['start'].get('dateTime', event['start'].get('date')),
            "location": event.get('location', 'N/A'),
        }
        formatted_events.append(formatted_event)

    return json.dumps({"date": date, "events": formatted_events})

In [22]:
def create_event(summary, start_time, end_time, location=None):
    """Create a new event in Google Calendar"""
    event = {
        'summary': summary,
        'start': {
            'dateTime': start_time,
            'timeZone': 'UTC',
        },
        'end': {
            'dateTime': end_time,
            'timeZone': 'UTC',
        },
    }

    if location:
        event['location'] = location

    created_event = calendar_service.events().insert(calendarId='primary', body=event).execute()
    return json.dumps({"event_id": created_event['id'], "summary": created_event['summary'], "status": "created"})

In [23]:
def delete_event(event_id):
    """Delete an event from Google Calendar"""
    calendar_service.events().delete(calendarId='primary', eventId=event_id).execute()
    return json.dumps({"event_id": event_id, "status": "deleted"})

In [24]:
def get_free_busy_info(start_time, end_time):
    """Get free/busy information from Google Calendar"""
    free_busy_query = {
        "timeMin": start_time,
        "timeMax": end_time,
        "items": [{"id": "primary"}],
    }

    free_busy_info = calendar_service.freebusy().query(body=free_busy_query).execute()
    return json.dumps(free_busy_info)

In [25]:
def update_event(event_id, summary=None, start_time=None, end_time=None, location=None):
    """Update an event in Google Calendar"""
    event = calendar_service.events().get(calendarId='primary', eventId=event_id).execute()

    if summary:
        event['summary'] = summary
    if start_time:
        event['start']['dateTime'] = start_time
    if end_time:
        event['end']['dateTime'] = end_time
    if location:
        event['location'] = location

    updated_event = calendar_service.events().update(calendarId='primary', eventId=event_id, body=event).execute()
    return json.dumps({"event_id": updated_event['id'], "summary": updated_event['summary'], "status": "updated"})


In [26]:
def reschedule_event(event_id, start_time, end_time):
    """Reschedule an event in Google Calendar"""
    return update_event(event_id, start_time=start_time, end_time=end_time)

In [27]:
# Function definitions for calendar-related queries
functions = [
    {
        "name": "get_calendar_events",
        "description": "Get the events on a given date from Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "date": {
                    "type": "string",
                    "description": "The date for which to retrieve events (in 'YYYY-MM-DD' format)",
                },
            },
            "required": ["date"],
        },
    },
    {
        "name": "create_event",
        "description": "Create a new event in Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": "The summary or title of the event",
                },
                "start_time": {
                    "type": "string",
                    "description": "The start time of the event (in RFC3339 format)",
                },
                "end_time": {
                    "type": "string",
                    "description": "The end time of the event (in RFC3339 format)",
                },
                "location": {
                    "type": "string",
                    "description": "The location of the event (optional)",
                },
            },
            "required": ["summary", "start_time", "end_time"],
        },
    },
    {
        "name": "delete_event",
        "description": "Delete an event from Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "event_id": {
                    "type": "string",
                    "description": "The ID of the event to be deleted",
                },
            },
            "required": ["event_id"],
        },
    },
    {
        "name": "get_free_busy_info",
        "description": "Get free/busy information from Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "start_time": {
                    "type": "string",
                    "description": "The start time for querying free/busy information (in RFC3339 format)",
                },
                "end_time": {
                    "type": "string",
                    "description": "The end time for querying free/busy information (in RFC3339 format)",
                },
            },
            "required": ["start_time", "end_time"],
        },
    },
    {
        "name": "update_event",
        "description": "Update an event in Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "event_id": {
                    "type": "string",
                    "description": "The ID of the event to be updated",
                },
                "summary": {
                    "type": "string",
                    "description": "The new summary or title of the event (optional)",
                },
                "start_time": {
                    "type": "string",
                    "description": "The new start time of the event (in RFC3339 format) (optional)",
                },
                "end_time": {
                    "type": "string",
                    "description": "The new end time of the event (in RFC3339 format) (optional)",
                },
                "location": {
                    "type": "string",
                    "description": "The new location of the event (optional)",
                },
            },
            "required": ["event_id"],
        },
    },
    {
        "name": "reschedule_event",
        "description": "Reschedule an event in Google Calendar",
        "parameters": {
            "type": "object",
            "properties": {
                "event_id": {
                    "type": "string",
                    "description": "The ID of the event to be rescheduled",
                },
                "start_time": {
                    "type": "string",
                    "description": "The new start time of the event (in RFC3339 format)",
                },
                "end_time": {
                    "type": "string",
                    "description": "The new end time of the event (in RFC3339 format)",
                },
            },
            "required": ["event_id", "start_time", "end_time"],
        },
    },
]


In [28]:
# Chat with the personalized calendar chatbot
context = [
    {"role": "user", "content": "What events do I have?"},
    {"role": "assistant", "content": 'I will check your calendar for events.'},
    {"role": "assistant", "content": 'Assistant to=functions.get_calendar_events'},
    {"role": "user", "content": "Tell me about my events."},
    {"role": "assistant", "content": "You don't have any events scheduled."},
    {"role": "user", "content": "Create a new event"},
    {"role": "assistant", "content": 'Assistant to=functions.create_event'},
    {"role": "user", "content": "Delete the event with ID XYZ"},
    {"role": "assistant", "content": 'Assistant to=functions.delete_event'},
    {"role": "user", "content": "Get free/busy information"},
    {"role": "assistant", "content": 'Assistant to=functions.get_free_busy_info'},
    {"role": "user", "content": "Update the event with ID XYZ"},
    {"role": "assistant", "content": 'Assistant to=functions.update_event'},
    {"role": "user", "content": "Reschedule the event with ID XYZ"},
    {"role": "assistant", "content": 'Assistant to=functions.reschedule_event'},
]

In [29]:
import json
import openai

def process_user_message(user_message, context, credentials):
    if user_message:
        # Append the user message to the conversation context
        context.append({"role": "user", "content": user_message})

        # Generate a response from the chatbot
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=context,
            functions=functions,
            max_tokens=100,
        )

        # Retrieve the assistant's reply from the response
        assistant_reply = response['choices'][0]['message']['content']

        # Check if the assistant's reply is not empty
        if assistant_reply:
            # Append the assistant's reply to the conversation context
            context.append({"role": "assistant", "content": assistant_reply})

            # Check if there's a function call in the response
            if response.choices[0].message.get("function_call"):
                fn_name = response.choices[0].message["function_call"]["name"]
                arguments = response.choices[0].message["function_call"]["arguments"]

                # Convert string to dictionary if necessary
                if isinstance(arguments, str):
                    arguments = json.loads(arguments)

                # Call the appropriate function
                if fn_name == "get_calendar_events":
                    response = get_calendar_events(**arguments)
                elif fn_name == "create_event":
                    # If 'arguments' is a string, parse the event details from it
                    if isinstance(arguments, str):
                        arguments = parse_event_details(arguments)
                    response = create_event(**arguments)  
                elif fn_name == "delete_event":
                    response = delete_event(**arguments)  
                elif fn_name == "update_event":
                    response = update_event(**arguments)
                elif fn_name == "reschedule_event":
                    response = reschedule_event(**arguments)

                # Append the function call and response to the conversation context
                context.append(
                    {
                        "role": "assistant",
                        "content": None,
                        "function_call": {"name": fn_name, "arguments": json.dumps(arguments)},
                    }
                )
                context.append(
                    {
                        "role": "function",
                        "name": fn_name,
                        "content": f"response: {str(response)}",
                    }
                )

        return assistant_reply, context
    else:
        return "", context

def parse_event_details(event_details):
    # Extract the event details from the user input
    event_args = re.findall(r'(?:"[^"]*"|\S)+', event_details)
    if len(event_args) >= 3:
        summary = event_args[0].strip('"')
        start_time = event_args[1].strip('"')
        end_time = event_args[2].strip('"')
        location = None
        if len(event_args) >= 4:
            location = event_args[3].strip('"')
        return {
            "summary": summary,
            "start_time": start_time,
            "end_time": end_time,
            "location": location,
        }
    else:
        return {} 

def main():
    # Set your OpenAI API key here (replace 'YOUR_API_KEY' with your actual API key)
    openai.api_key = "YOUR API KEY"

    # Initialize the conversation context as an empty list
    conversation_context = []

    print("Bot: Hi! I'm your chatbot assistant. How can I help you today?")

    while True:
        user_input = get_user_message()

        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Bot: Goodbye!")
            break

        # Process the user message and get the assistant's reply
        assistant_reply, conversation_context = process_user_message(
            user_input, conversation_context, credentials
        )

        # Display the assistant's reply
        print("Bot:", assistant_reply)

if __name__ == "__main__":
    main()

Bot: Hi! I'm your chatbot assistant. How can I help you today?
You: Can you create event?
Bot: None
You: Can you create an event for me?
Bot: Sure! I can create an event for you. Please provide me with the necessary details for the event.
You: What do you want?
Bot: To create an event, I would need the following details:

1. Summary or title of the event.
2. Start time and date of the event.
3. End time and date of the event.
4. Location of the event (optional).

Please provide me with these details so that I can create the event for you.
You: Title is Meeting with the team
Bot: Thank you for providing the title of the event. 

Now, please provide me with the start time and date of the event in RFC3339 format. For example, "2022-01-01T09:00:00Z" represents January 1, 2022, at 09:00 AM UTC.
You: start date is 2024-01-01T09:00:00Z
Bot: Thank you for providing the start date and time of the event. 

Now, please provide me with the end time and date of the event in RFC3339 format.
You: 202